### LLAMA 3 Finetune

SFT for following 3 indian languages + english language.

1. Hindi
2. Telugu
3. Tamil
4. English

We use [unsloth](https://github.com/unslothai/unsloth) library for fine-tuning the `llama3-8b` model.

### Installation

In [3]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate 
!pip install --no-deps xformers trl peft  bitsandbytes wandb -U

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-nhqnfbgh/unsloth_7ddde97275204d10baebacc6b310a4db
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-nhqnfbgh/unsloth_7ddde97275204d10baebacc6b310a4db

  Resolved https://github.com/unslothai/unsloth.git to commit a106e0b256fb9cdb2e423eddd9edef805f292c52
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.5 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.3 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.5.8-cp310-cp310-linux_x86_64.whl size=120853537 sha256=53979129f883680327bf5d13027cd014e2d054f4

### wandb login

In [4]:
!wandb login ''

wandb: Appending key for api.wandb.ai to your netrc file: /teamspace/studios/this_studio/.netrc


### HuggingFace login

In [35]:
from huggingface_hub import notebook_login

# This will prompt you to enter your token
notebook_login()

### Download Model

In [ ]:
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit", # Instruct version of Gemma 7b
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit", # Instruct version of Gemma 2b
    "unsloth/llama-3-8b-bnb-4bit", # [NEW] 15 Trillion token Llama-3
]

In [44]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = torch.bfloat16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use for 4bit quantization

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", 
    # model_name="unsloth/gemma-7b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = '',
)

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: NVIDIA A10G. Max memory: 22.191 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### LoRA setup

In [39]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = 'unsloth',
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


### Test tokens for each language

In [8]:
# English reference text

text = """Google News is a news aggregator service developed by Google. It presents a continuous flow of links to articles organized from thousands of publishers and magazines. Google News is available as an app on Android, iOS, and the Web. Google released a beta version in September 2002 and the official app in January 2006."""

print(text)

Google News is a news aggregator service developed by Google. It presents a continuous flow of links to articles organized from thousands of publishers and magazines. Google News is available as an app on Android, iOS, and the Web. Google released a beta version in September 2002 and the official app in January 2006.


In [9]:
hindi_text = """Google News Google द्वारा विकसित एक समाचार एग्रीगेटर सेवा है। यह हजारों प्रकाशकों और पत्रिकाओं से व्यवस्थित लेखों के लिंक का निरंतर प्रवाह प्रस्तुत करता है। Google News एंड्रॉइड, iOS और वेब पर एक ऐप के रूप में उपलब्ध है। Google ने सितंबर 2002 में बीटा संस्करण और जनवरी 2006 में आधिकारिक ऐप जारी किया।"""
tamil_text = """கூகுள் செய்திகள் என்பது கூகுள் உருவாக்கிய செய்தி சேகரிப்பு சேவையாகும். ஆயிரக்கணக்கான வெளியீட்டாளர்கள் மற்றும் பத்திரிகைகளிலிருந்து ஒழுங்கமைக்கப்பட்ட கட்டுரைகளுக்கான தொடர்ச்சியான இணைப்புகளை இது வழங்குகிறது. Android, iOS மற்றும் இணையத்தில் Google செய்திகள் ஒரு பயன்பாடாகக் கிடைக்கிறது. கூகுள் செப்டம்பர் 2002 இல் பீட்டா பதிப்பையும், ஜனவரி 2006 இல் அதிகாரப்பூர்வ பயன்பாட்டையும் வெளியிட்டது."""
telugu_text = """Google వార్తలు అనేది Google ద్వారా అభివృద్ధి చేయబడిన వార్తా అగ్రిగేటర్ సేవ. ఇది వేలకొద్దీ ప్రచురణకర్తలు మరియు మ్యాగజైన్‌ల నుండి నిర్వహించబడిన కథనాలకు నిరంతర లింక్‌లను అందిస్తుంది. Google వార్తలు Android, iOS మరియు వెబ్‌లో యాప్‌గా అందుబాటులో ఉన్నాయి. గూగుల్ సెప్టెంబరు 2002లో బీటా వెర్షన్‌ను మరియు జనవరి 2006లో అధికారిక యాప్‌ను విడుదల చేసింది."""

In [40]:
language_text = {
         'hindi': hindi_text, 
         'tamil': tamil_text, 
         'telugu': telugu_text, 
        }

for language in language_text:
    print("Language: ", language)
    print(tokenizer.tokenize(language_text[language][:50]))
    print("------------------------------------------------------------")

Language:  hindi
['Google', '▁News', '▁Google', '▁द्वारा', '▁विक', 'स', 'ित', '▁एक', '▁समा', 'चार', '▁ए', 'ग्री', 'गे']
------------------------------------------------------------
Language:  tamil
['க', 'ூ', 'கு', 'ள்', '▁செய', '்த', 'ிகள்', '▁என்ப', 'து', '▁கூ', 'கு', 'ள்', '▁உ', 'ரு', 'வா', 'க்க', 'ிய', '▁செய', '்த', 'ி', '▁ச']
------------------------------------------------------------
Language:  telugu
['Google', '▁వ', 'ార్', 'త', 'లు', '▁అ', 'నే', 'ది', '▁Google', '▁ద', '్వ', 'ార', 'ా', '▁అ', 'భ', 'ి', 'వ', 'ృ', 'ద్', 'ధ', 'ి', '▁చే', 'య', 'బ', 'డ']
------------------------------------------------------------


In [11]:
from datasets import load_dataset, concatenate_datasets, Dataset
import json
import pandas as pd

### Prompt

In [12]:
alpaca_prompt = """
### Instruction:
{}

### Input:
{}

### Response:
{}"""

### EOS_TOKEN

In [13]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

### Prompts 

In [14]:
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        if input is None:
            input = ""
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

def formatting_prompts_func_telugu(examples):
    instructions = examples["telugu_instruction"]
    inputs       = examples["telugu_input"]
    outputs      = examples["telugu_output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        if input is None:
            input = ""
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

In [15]:
def process_dataset(dataset_name, 
                    split_type, 
                    processing_func, 
                    rename_column = False, 
                    post_processing_steps=False,
                    filter_data = False, 
                    column_name = 'id', 
                    value = 'alpaca',
                    num_samples=20000):
    
    if isinstance(dataset_name, str):
        dataset = load_dataset(dataset_name, split=split_type)
    else:
        # Assuming dataset_name is a filepath for JSON file
        with open(dataset_name, 'r') as file:
            data = []
            for line_number, line in enumerate(file, 1):
                try:
                    data.append(json.loads(line))
                except json.JSONDecodeError as e:
                    print(f"Error parsing JSON at line {line_number}: {e}")
            dataset = Dataset.from_pandas(pd.DataFrame(data))
    
    if post_processing_steps:
        for post_processing_step in post_processing_steps:
            dataset = post_processing_step(dataset)
    
    if rename_column:
        dataset = rename(dataset)
    
    if filter_data:
        dataset = filter_dataset(dataset, num_samples, value, column_name)
    
    dataset = dataset.map(processing_func, batched=True)
    
    return dataset

# Define the additional processing steps
def rename(dataset):
    return dataset.rename_column('response', 'output')

def filter_dataset(dataset, num_samples, value, column_name):
    return dataset.filter(lambda example: value in example[column_name]).shuffle(seed=42).select(range(num_samples))

### Process each language dataset

#### English

In [42]:
dataset_en = process_dataset("yahma/alpaca-cleaned", "train", formatting_prompts_func)

print(dataset_en['text'][0])


### Instruction:
Give three tips for staying healthy.

### Input:


### Response:
1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.

2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.

3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.<|end_of_text|>


#### Hindi

In [17]:
dataset_hi1 = process_dataset("ravithejads/samvaad-hi-filtered", "train", formatting_prompts_func)

dataset_hi2 = process_dataset("HydraIndicLM/hindi_alpaca_dolly_67k", "train", formatting_prompts_func, filter_data = True)

print(dataset_hi1['text'][0])

print(dataset_hi2['text'][0])


### Instruction:
नीचे एक निर्देश है जो आपको बताता है कि किसी कार्य को कैसे पूरा किया जाए। ऐसा उत्तर लिखें जो अनुरोध को पर्याप्त रूप से पूरा करता हो।

### Input:
वैश्वीकरण भारत में स्वदेशी संस्कृतियों और भाषाओं के संरक्षण के लिए क्या चुनौती पेश करता है?

### Response:
वैश्वीकरण भारत में स्वदेशी संस्कृतियों और भाषाओं के संरक्षण के लिए कई चुनौतियों का सामना कर रहा है। एक महत्वपूर्ण चुनौती वैश्विक मीडिया और लोकप्रिय संस्कृति का प्रभाव है, जो अक्सर अंग्रेजी और हिंदी जैसी प्रमुख भाषाओं को बढ़ावा देता है, जिससे स्वदेशी भाषाओं का महत्व कम हो जाता है। इससे स्वदेशी समुदायों के भीतर मूल भाषाओं के अंतर-पीढ़ीगत संचरण में गिरावट आ सकती है।

इसके अतिरिक्त, आर्थिक वैश्वीकरण से उपभोक्तावाद और समरूप जीवन शैली का प्रसार हो सकता है, जो पारंपरिक सांस्कृतिक प्रथाओं और मूल्यों को नष्ट कर सकता है। नतीजतन, स्वदेशी समुदायों के भीतर युवा पीढ़ियाँ जीवन जीने के आधुनिक तरीकों को अपनाने के लिए अधिक इच्छुक हो सकती हैं, जिससे वे अपनी विरासत से और दूर हो सकते हैं।

इसके अलावा, बेहतर अवसरों की तलाश में शहरी केंद्रों मे

#### Tamil

In [18]:
print("Language: ")

dataset_ta = process_dataset("abhinand/tamil-alpaca", "train", formatting_prompts_func)

print(dataset_ta['text'][0])

Language: 



### Instruction:
ஆரோக்கியமாக இருப்பதற்கு மூன்று குறிப்புகளைக் கொடுங்கள்.

### Input:


### Response:
1. சமச்சீரான மற்றும் சத்தான உணவை உண்ணுங்கள்: உங்கள் உணவில் பலவகையான பழங்கள் மற்றும் காய்கறிகள், மெலிந்த புரதம், முழு தானியங்கள் மற்றும் ஆரோக்கியமான கொழுப்புகள் உள்ளன என்பதை உறுதிப்படுத்திக் கொள்ளுங்கள். இது உங்கள் உடலைச் சிறப்பாகச் செயல்பட தேவையான ஊட்டச்சத்துக்களுடன் வழங்க உதவுகிறது மற்றும் நாள்பட்ட நோய்களைத் தடுக்க உதவுகிறது.

2. வழக்கமான உடல் செயல்பாடுகளில் ஈடுபடுங்கள்: வலுவான எலும்புகள், தசைகள் மற்றும் இருதய ஆரோக்கியத்தை பராமரிக்க உடற்பயிற்சி முக்கியமானது. ஒவ்வொரு வாரமும் குறைந்தபட்சம் 150 நிமிட மிதமான ஏரோபிக் உடற்பயிற்சி அல்லது 75 நிமிட தீவிரமான உடற்பயிற்சியை இலக்காகக் கொள்ளுங்கள்.

3. போதுமான தூக்கத்தைப் பெறுங்கள்: போதுமான தரமான தூக்கம் உடல் மற்றும் மன நலனுக்கு முக்கியமானது. இது மனநிலையை சீராக்க உதவுகிறது, அறிவாற்றல் செயல்பாட்டை மேம்படுத்துகிறது மற்றும் ஆரோக்கியமான வளர்ச்சி மற்றும் நோயெதிர்ப்பு செயல்பாட்டை ஆதரிக்கிறது. ஒவ்வொரு இரவும் 7-9 மணிநேர தூக்கத்தை இலக்காகக் கொள்ளுங்கள்.<|en

#### Telugu

In [19]:
dataset_te1 = process_dataset("Telugu-LLM-Labs/yahma_alpaca_cleaned_telugu_filtered_and_romanized", "train", formatting_prompts_func_telugu)

dataset_te2 = process_dataset("Telugu-LLM-Labs/teknium_GPTeacher_general_instruct_telugu_filtered_and_romanized", "train", formatting_prompts_func_telugu)

print(dataset_te1['text'][0])

print(dataset_te2['text'][0])


### Instruction:
ఆరోగ్యంగా ఉండటానికి మూడు చిట్కాలు ఇవ్వండి.

### Input:


### Response:
1. సమతుల్య మరియు పోషకమైన ఆహారం తినండి: మీ భోజనంలో వివిధ రకాల పండ్లు మరియు కూరగాయలు, సన్నని ప్రోటీన్, తృణధాన్యాలు మరియు ఆరోగ్యకరమైన కొవ్వులు ఉన్నాయని నిర్ధారించుకోండి. ఇది మీ శరీరానికి ఉత్తమంగా పనిచేయడానికి అవసరమైన పోషకాలను అందించడంలో సహాయపడుతుంది మరియు దీర్ఘకాలిక వ్యాధులను నివారించడంలో సహాయపడుతుంది.

2. క్రమం తప్పకుండా శారీరక శ్రమలో పాల్గొనండి: బలమైన ఎముకలు, కండరాలు మరియు హృదయ ఆరోగ్యాన్ని నిర్వహించడానికి వ్యాయామం కీలకం. ప్రతి వారం కనీసం 150 నిమిషాల మితమైన ఏరోబిక్ వ్యాయామం లేదా 75 నిమిషాల తీవ్రమైన వ్యాయామం లక్ష్యంగా పెట్టుకోండి.

3. తగినంత నిద్ర పొందండి: శారీరక మరియు మానసిక శ్రేయస్సుకు తగినంత నాణ్యమైన నిద్ర పొందడం చాలా ముఖ్యం. ఇది మానసిక స్థితిని నియంత్రించడానికి, అభిజ్ఞా పనితీరును మెరుగుపరచడానికి మరియు ఆరోగ్యకరమైన పెరుగుదల మరియు రోగనిరోధక పనితీరుకు మద్దతు ఇస్తుంది. ప్రతి రాత్రి 7-9 గంటల నిద్రను లక్ష్యంగా పెట్టుకోండి.<|end_of_text|>

### Instruction:
మానవ పరిణామ ప్రక్రియ యొక్క అవలోకనాన్ని అందించండి.

In [20]:
# dataset_en.select(range(1000))

In [43]:
dataset_en

Dataset({
    features: ['output', 'input', 'instruction', 'text'],
    num_rows: 51760
})

In [21]:
dataset_en =dataset_en.select(range(1000))
dataset_hi1 =dataset_hi1.select(range(1000))
dataset_hi2 =dataset_hi2.select(range(1000))
dataset_ta =dataset_ta.select(range(1000))
dataset_te1 =dataset_te1.select(range(1000))
dataset_te2 =dataset_te2.select(range(1000))

### Final dataset concatenation

In [22]:
dataset = concatenate_datasets([
    dataset_en, dataset_hi1, dataset_hi2,
     dataset_ta, dataset_te1, dataset_te2
])

### Shuffle dataset

In [23]:
print(dataset['text'][100])


### Instruction:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Previously, the use of dye-sub printing was limited to industrial or high-end commercial printing. Dye-sub photo printing has been used in medical imaging, graphic arts proofing, security, and broadcast-related applications. Today, it is extremely popular in event photography and photo booths or kiosks that require high-speed, on-demand printing.

Alps Electric produced the first quality dye-sub printers for home consumers in the $500–$1,000 price range, bringing dye-sublimation technology within the reach of a wider audience. (These models were, however, not true page printers, since they used a narrow printhead that swept across the page, like most inkjet printers.) Now there are many dye-sublimation printers on the market starting from as low as $100, especially postcard-sized mobile photo printers.

The

In [24]:
dataset_sft = dataset.shuffle(seed=42)

In [25]:
print(dataset_sft['text'][100])


### Instruction:
பின்வரும் பட்டியலை இறங்கு வரிசையில் மறுசீரமைக்கவும்.

### Input:
9, -7, 15, 12

### Response:
15, 12, 9, -7<|end_of_text|>


In [26]:
dataset

Dataset({
    features: ['output', 'input', 'instruction', 'text', 'id', 'system_prompt', 'telugu_instruction', 'telugu_input', 'telugu_output', 'telugu_transliterated_instruction', 'telugu_transliterated_input', 'telugu_transliterated_output'],
    num_rows: 6000
})

### Count tokens

In [27]:
tokens = tokenizer.tokenize(dataset_sft['text'][100])
len(tokens)

104

### Hyper parameters

In [28]:
# # Output directory where the model predictions and checkpoints will be stored
# output_dir = "./results"

# # Number of training epochs
# num_train_epochs = 2

# # Enable fp16/bf16 training (set bf16 to True with an A100)
# fp16 = False
# bf16 = True

# # Batch size per GPU for training
# per_device_train_batch_size = 8

# # Batch size per GPU for evaluation
# per_device_eval_batch_size = 8

# # Number of update steps to accumulate the gradients for
# gradient_accumulation_steps = 8

# # Enable gradient checkpointing
# gradient_checkpointing = True

# # Maximum gradient normal (gradient clipping)
# max_grad_norm = 1.0

# # Initial learning rate (AdamW optimizer)
# learning_rate = 2e-4

# # Weight decay to apply to all layers except bias/LayerNorm weights
# weight_decay = 0.001

# # Optimizer to use
# optim = "paged_adamw_32bit"

# # Learning rate schedule
# lr_scheduler_type = "cosine"

# # Number of training steps (overrides num_train_epochs)
# max_steps = -1

# # Ratio of steps for a linear warmup (from 0 to learning rate)
# warmup_ratio = 0.03

# # Group sequences into batches with same length
# # Saves memory and speeds up training considerably
# group_by_length = True

# # Save checkpoint every X updates steps
# save_steps = 0

# # Log every X updates steps
# logging_steps = 10

# # Pack multiple short examples in the same input sequence to increase efficiency and make training 5x faster for short sequences.
# packing = True

# # text field in dataset
# dataset_text_field = "text"

# # dataset para,
# dataset_num_proc = 2

# # Load the entire model on the GPU 0
# # device_map = {"": 0}
# device_map = "auto"

# # monitoring
# report_to = "wandb"

### TrainingArguments

In [29]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "results",
    ),
)

max_steps is given, it will override any value given in num_train_epochs


### Current memory stats

In [30]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A10G. Max memory = 22.191 GB.
5.594 GB of memory reserved.


### Monitoring

In [31]:
import wandb
wandb.init(project="LLAMA_3_SOUTH", name="LLAMA_3_SOUTH")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: macharlasaiteja. Use `wandb login --relogin` to force relogin


### Start training

In [32]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 6,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 8
\        /    Total batch size = 64 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
1,0.751600
2,0.813600
3,0.686400
4,0.808100
5,0.740800
6,0.733000
7,0.571600
8,0.674700
9,0.723100
10,0.676500


### Save model

In [33]:
model.save_pretrained("LLAMA_3_SOUTH") # Local saving
model.push_to_hub("/LLAMA_3_SOUTH", token = '') 
tokenizer.push_to_hub("/LLAMA_3_SOUTH", token = '') 

README.md:   0%|          | 0.00/574 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/Saiteja/LLAMA_3_SOUTH


### Test Query

In [34]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "(9+1)+(5+0). इसे 3 चरणों में हल करें.", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 300, use_cache = True)
print(tokenizer.batch_decode(outputs))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['<|begin_of_text|>\n### Instruction:\n(9+1)+(5+0). इसे 3 चरणों में हल करें.\n\n### Input:\n\n\n### Response:\n1. 9+1 = 10\n2. 5+0 = 5\n3. 10+5 = 15<|end_of_text|>']


In [46]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "मुझे खगोल विज्ञान के बारे में बताओ.", # instruction
        "provide Oupput in Telugu", 
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 400, use_cache = True)
print(tokenizer.batch_decode(outputs))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['<|begin_of_text|>\n### Instruction:\nमुझे खगोल विज्ञान के बारे में बताओ.\n\n### Input:\nprovide Oupput in Telugu\n\n### Response:\nనేను క్షీణమైన గ్రహాన్ని చూస్తాను. ఇది ఒక గ్రహం లేదా సూర్యకుండలో చూస్తాను. నేను వీరిని కలిస్తాను. నేను చూస్తాను. నేను చూస్తాను. నేను చూస్తాను. నేను చూస్తాను. నేను చూస్తాను. నేను చూస్తాను. నేను చూస్తాను. నేను చూస్తాను. నేను చూస్తాను. నేను చ']
